In [ ]:
import machine
import dht
from microWebSrv import MicroWebSrv  
import json
import analogio

# DHT22 sensor
dht_sensor = dht.DHT22(machine.Pin(2))

# MQ131 sensor 
mq = analogio.AnalogIn(machine.ADC(0))

# MQ calibration values 
MQ_MIN = 9000
MQ_MAX = 40000

def _httpHandlerDHTGet(_httpClient_, _httpResponse_):

  dht_sensor.measure()
  temp = dht_sensor.temperature()/10
  humid = dht_sensor.humidity()/10

  # Read MQ131
  mq_raw = mq.value
  
  # Convert MQ value to percentage
  mq_perc = (mq_raw - MQ_MIN) / (MQ_MAX - MQ_MIN) * 100
  mq_perc = max(0, min(100, mq_perc))

  data = {'air_quality': mq_perc,'temperature':temp,'humidity':humid}
  
  print(data)

  # Return JSON response
  _httpResponse_.WriteResponseOk(
     headers = {'Cache-Control': 'no-cache'},
     contentType = "text/event-stream",
     contentCharset = "UTF-8",
     content = 'data: {0}\n\n'.format(json.dumps(data))
  )

routeHandlers = [('/dht','GET', _httpHandlerDHTGet)]

srv = MicroWebSrv(_routeHandlers_=routeHandlers,_webPath_='/www/')
srv.Start(_threaded_=False)